In [1]:
from exportaciones import EXPORTACIONES

In [2]:
xs = EXPORTACIONES()

xs.donwload_data()

In [ ]:
xs.group_by_(["entidad"])

In [4]:
exp = xs.filter()

32

In [96]:
def gen_nacional(df):
    df = df[(df["year"]>"2020") & (df["year"]<"2023")]

    df_montos = (df.groupby(["year"]).agg({"monto":"sum"}))

    monto = df_montos.loc["2022", "monto"]
    crecimiento = (df_montos.pct_change()*100).loc["2022", "monto"]

    ranking = (df[df["year"] == "2022"]
               .groupby("entidad").agg({"monto":"sum"})
               .rank(ascending=False)
               .sort_values("monto"))

    return [monto, crecimiento, ranking]

gen_nacional(exp)[2]

,monto
entidad,
Chihuahua,1.0
Coahuila de Zaragoza,2.0
Baja California,3.0
Nuevo León,4.0
Tamaulipas,5.0
Guanajuato,6.0
Jalisco,7.0
Sonora,8.0
México,9.0


In [105]:
def gen_entidad(df, entidad):
    monto_nacional = gen_nacional(df)[1]
    ranking = gen_nacional(df)[2]

    df = exp[exp["entidad"] == entidad].reset_index()

    df = df[(df["year"]>"2020") & (df["year"]<"2023")]
    df = df.groupby(["year"]).agg({"monto":"sum"})
    
    monto_estatal = round(df.loc["2022", "monto"],2)
    porcentaje_estatal = round((monto_nacional/monto_estatal)*100, 1)

    crecimiento = (df.groupby(["year"]).agg({"monto":"sum"})
                   .pct_change()*100).loc["2022","monto"]
    
    r_estatal = ranking[ranking.index == entidad].loc[entidad, "monto"]

    return [monto_estatal,porcentaje_estatal, crecimiento, r_estatal]

gen_entidad(exp, "Hidalgo")

[3303.29, 0.5, 78.72478566257668, 21.0]

In [126]:
def gen_group_nacional(df):
    df = df[df["year"] == "2022"]
    df = df.groupby(["subsector"]).agg({"monto":"sum"})

    return df

gen_group_nacional(exp)

,monto
subsector,
Agricultura,15496.072
Cría y explotación de animales,1004.455
"Curtido y acabado de cuero y piel, y fabricación de productos de cuero, piel y materiales sucedáneos",1310.176
Extracción de petróleo y gas,31865.120
"Fabricación de accesorios, aparatos eléctricos y equipo de generación de energía eléctrica",31226.293
"Fabricación de equipo de computación, comunicación, medición y de otros equipos, componentes y accesorios electrónicos",92253.000
Fabricación de equipo de transporte,185242.807
Fabricación de insumos textiles y acabado de textiles,936.513
Fabricación de maquinaria y equipo,23664.756


In [131]:
def group_entidad(df, entidad):
    df = df[(df["year"] == "2022") & (df["entidad"] == entidad)]

    df = df.groupby("subsector").agg({"monto":"sum"})
    df = df.merge(gen_group_nacional(exp), left_index = True, right_index=True)

    df["%"] = (df["monto_x"]/df["monto_y"])*100
    df.columns = ["Exportaciones entidad", "Exportaciones nacional", "%"]
    df = df.sort_values("Exportaciones entidad", ascending = False)
    
    return df

group_entidad(exp, "Aguascalientes")

,Exportaciones entidad,Exportaciones nacional,%
subsector,,,
Fabricación de equipo de transporte,9601.577,185242.807,5.183239
"Fabricación de equipo de computación, comunicación, medición y de otros equipos, componentes y accesorios electrónicos",1185.650,92253.000,1.285216
Fabricación de prendas de vestir,213.736,3348.879,6.382315
Industria alimentaria,209.912,15510.212,1.353379
Fabricación de maquinaria y equipo,143.427,23664.756,0.606079
Fabricación de productos metálicos,132.982,13618.471,0.976483
Subsectores no especificados,108.662,8186.216,1.327378
Agricultura,44.851,15496.072,0.289435
Industria del plástico y del hule,36.392,14457.166,0.251723
